In [1]:
import pandas as pd
import geopandas as gpd
import os
# user = 'aolsen'

In [2]:
bayareafips_full = {'06001': 'Alameda', '06013': 'Contra Costa', '06041': 'Marin', '06055': 'Napa',
                    '06075': 'San Francisco', '06081': 'San Mateo', '06085': 'Santa Clara',
                    '06097': 'Sonoma', '06095': 'Solano'}

In [3]:
PBA50_output_folder = 'C:/Users/{}/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50'.format(os.getenv('USERNAME'))
validation_folder = 'M:/Data/Urban/BAUS/pba50plus_validation'

# Load data

In [4]:
# tazs
ZONES_PATH = 'C:/Users/{}/Box/Modeling and Surveys/Urban Modeling/Spatial/Zones/TAZ1454/zones1454.shp'.format(os.getenv('USERNAME'))

zones = gpd.read_file(ZONES_PATH).to_crs('EPSG:26910')
zones['geom_pt'] = zones.representative_point()

In [5]:
superdistricts = zones.dissolve('superdistr')

In [6]:
# get mapping - surely we have this somewhere in a text file

superdistrict_to_county_map = zones.groupby(
    ['superdistr', 'fipsstco']).size().reset_index(1).fipsstco.map(bayareafips_full)

## Observed data / Shimon

In [7]:
taz_2020 = pd.read_csv(
    'https://raw.githubusercontent.com/BayAreaMetro/petrale/main/applications/travel_model_lu_inputs/2020/TAZ1454_2020_long.csv')
taz_2023 = pd.read_csv(
    'https://raw.githubusercontent.com/BayAreaMetro/petrale/main/applications/travel_model_lu_inputs/2023/TAZ1454_2023_long.csv')

In [8]:
# taz_2020['SD'] = taz_2020.ZONE.map(zones.set_index('taz1454').superdistr)
# taz_2023['SD'] = taz_2023.ZONE.map(zones.set_index('taz1454').superdistr)

## PBA 50 data - taz

In [9]:
tazdata_2010 = pd.read_csv(
    os.path.join(PBA50_output_folder, 
                 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182_taz_summaries_2010.csv'))

tazdata_2010['superdistrict'] = tazdata_2010.SD

In [10]:
tazdata_2020 = pd.read_csv(
    os.path.join(PBA50_output_folder, 
                 'Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182_taz_summaries_2020.csv'))
tazdata_2020['superdistrict'] = tazdata_2020.SD

## Census 2010

In [11]:
census2010_block = gpd.read_file('C:/Users/{}/Box/Modeling and Surveys/Census/censusblocks2010_w_dat.gpkg'.format(os.getenv('USERNAME')))
census2010_block['geom_pt'] = census2010_block.representative_point()

In [12]:
# assign containing zone to census block
bayarea_blocks_2010_x_zones = gpd.sjoin(
    census2010_block[['GEOID10','geom_pt']].set_geometry('geom_pt'), zones)


In [13]:
census2010_block['zone_id'] = census2010_block.GEOID10.map(bayarea_blocks_2010_x_zones.groupby(['GEOID10']).taz1454.first())
census2010_block['superdistrict'] = census2010_block.GEOID10.map(bayarea_blocks_2010_x_zones.groupby(['GEOID10']).superdistr.first())

## Census 2020

In [14]:
census2020_block = gpd.read_file('C:/Users/{}/Box/Modeling and Surveys/Census/censusblocks2020_w_dat.gpkg'.format(os.getenv('USERNAME')))

In [15]:
#census_2020_data = pd.read_excel('/Users/{user}/Downloads/census_2020_dhc_block_vars.xlsx')

In [16]:
census2020_block['superdistrict'] = census2020_block.zone_id.map(zones.set_index('taz1454').superdistr)

In [17]:
census2010_block.groupby(['superdistrict'])[['population','households']].sum().head()

,population,households
superdistrict,,
1.0,139649,75771
2.0,206581,101971
3.0,324381,118333
4.0,134624,49736
5.0,290434,98382


In [18]:
valid_2020 = pd.concat([
    tazdata_2010.groupby(['superdistrict']).TOTHH.sum(),
    tazdata_2020.groupby(['superdistrict']).TOTHH.sum(),
    census2010_block.groupby(['superdistrict']).households.sum(),
    census2020_block.groupby(['superdistrict']).households.sum(),
    taz_2020.query('Variable=="TOTHH" & Year==2020').groupby(
        ['DISTRICT']).Value.sum(),
    taz_2023.query('Variable=="TOTHH" & Year==2023').groupby(
        ['DISTRICT']).Value.sum()
],  # check, then double check the ordering matches the above list
    keys=['baseyear2010_BAUS', 
          'modeled2020_BAUS',
          'observed_census2010',
          'observed_census2020',
          'observed_acs2020',
          'estim_acs2023'],
    names=['what'])
valid_2020

what               superdistrict
baseyear2010_BAUS  1                 80012.0
                   2                105018.0
                   3                116392.0
                   4                 48566.0
                   5                 94463.0
                                      ...   
estim_acs2023      30                92804.0
                   31                32297.0
                   32                23258.0
                   33                42563.0
                   34                37862.0
Length: 204, dtype: float64

# Comparisons

In [19]:
# 2020 observations

valid_2020 = pd.concat([
    tazdata_2010.groupby(['superdistrict']).TOTHH.sum(),
    tazdata_2020.groupby(['superdistrict']).TOTHH.sum(),
    census2010_block.groupby(['superdistrict']).households.sum(),
    census2020_block.groupby(['superdistrict']).households.sum(),
    taz_2020.query('Variable=="TOTHH" & Year==2020').groupby(
        ['DISTRICT']).Value.sum(),
    taz_2023.query('Variable=="TOTHH" & Year==2023').groupby(
        ['DISTRICT']).Value.sum()
],  # check, then double check the ordering matches the above list
    keys=['baseyear2010_BAUS', 
          'modeled2020_BAUS',
          'observed_census2010',
          'observed_census2020',
          'observed_acs2020',
          'estim_acs2023'],
    names=['what'])
valid_2020 = valid_2020.unstack([0])  # .reset_index(name='value')

In [20]:
# store comparison expressions in a dict for better readability

comparisons = {
    # model growth relative to *census* 2010
    'ch_BAUS_minus_c2010': 'modeled2020_BAUS - observed_census2010',
    # model growth relative to *model* 2010
    'ch_BAUS2020_minus_BAUS2010': 'modeled2020_BAUS - baseyear2010_BAUS',

    # growth 2010-2020 per census data
    'ch_c2020_minus_c2010': 'observed_census2020 - observed_census2010',

    # growth 2010-2023 per census data
    'ch_acs2023_minus_c2010': 'estim_acs2023 - observed_census2010',

    # ratio of model growth to observed growth
    'rat_modelgr_vs_obsgr': 'ch_BAUS2020_minus_BAUS2010 / ch_c2020_minus_c2010'}

In [21]:
# Mean average percentage error

# manual approach
mean_MAPE = ((valid_2020.modeled2020_BAUS - valid_2020.observed_census2020) / \
    valid_2020.observed_census2020)
print(mean_MAPE.mean())

# vectorized approach
model_var = 'modeled2020_BAUS'
reference_var = 'observed_census2020'
print(valid_2020[[reference_var,model_var]].pct_change(axis=1)[model_var].mean())

-0.0017301432654833679
-0.0017301432654833586


In [22]:
# turn into df
valid_2020_df = valid_2020.reset_index()
valid_2020_df['county'] = valid_2020_df.superdistrict.map(
    superdistrict_to_county_map)
valid_2020_df['geometry'] = valid_2020_df.superdistrict.map(
    superdistricts.geometry)
valid_2020_df = valid_2020_df.set_geometry('geometry')

In [23]:
# loop through comparison expresions; assign to dataframe
for var, expr in comparisons.items():
    print(var, expr)
    valid_2020_df[var] = valid_2020_df.eval(expr)
    valid_2020_df[var]

ch_BAUS_minus_c2010 modeled2020_BAUS - observed_census2010
ch_BAUS2020_minus_BAUS2010 modeled2020_BAUS - baseyear2010_BAUS
ch_c2020_minus_c2010 observed_census2020 - observed_census2010
ch_acs2023_minus_c2010 estim_acs2023 - observed_census2010
rat_modelgr_vs_obsgr ch_BAUS2020_minus_BAUS2010 / ch_c2020_minus_c2010


In [24]:
# add run_id
valid_2020_df['run_id'] = 'PBA50-FBP'

# Output
Write out to tableau

In [25]:
# only need to write out the spatial file once
geo_out_path = os.path.join(validation_folder, 'baus_2020_validation_geo.geojson')

valid_2020_df[['superdistrict', 'geometry']].to_file(geo_out_path,driver='GeoJSON')


In [26]:
# write out the validation metrics for one model run
csv_out_path = os.path.join(validation_folder, 'baus_2020_validation_{}.csv'.format('PBA50-FBP'))

valid_2020_df.drop(['geometry'], axis=1).to_csv(csv_out_path, index=False)

In [27]:
print(list
(valid_2020_df))

['superdistrict', 'baseyear2010_BAUS', 'modeled2020_BAUS', 'observed_census2010', 'observed_census2020', 'observed_acs2020', 'estim_acs2023', 'county', 'geometry', 'ch_BAUS_minus_c2010', 'ch_BAUS2020_minus_BAUS2010', 'ch_c2020_minus_c2010', 'ch_acs2023_minus_c2010', 'rat_modelgr_vs_obsgr', 'run_id']
